In [745]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn import metrics


import pandas as pd
import numpy as np



from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [746]:
import nltk
import string
sw = nltk.corpus.stopwords.words('english')



In [747]:
punc = list(string.punctuation)

In [748]:
for word in punc:
    sw.append(word)
sw.append('br')
    
stopwords = set(sw)
    

In [749]:
def read_file(filename):
    reviews = []

    with open(filename, 'r') as f:
        t = f.readlines()
    for line in t:
        ids, rating, review = line.split(',', 2)
        reviews.append([ids, rating, review.strip().lower()])
    df = pd.DataFrame(reviews, columns=['id', 'rating', 'review'])
    df['rating'] = df.rating.astype("int")
    return df
    


In [750]:
pos = read_file('./pos_train_reviews.csv')
pos['sent'] = 1
pos['rating'] = pos.rating.astype("int")
pos.loc[pos.rating == 0, 'rating'] = 10


neg = read_file('./neg_train_reviews.csv')
neg['sent'] = 0

In [751]:
train = pos.append(neg)

In [754]:
pos.shape

(12500, 4)

In [8]:
test_pos = read_file('./pos_test_reviews.csv')
test_pos['sent'] = 1
test_pos['rating'] = test_pos.rating.astype("int")
test_pos.loc[test_pos.rating == 0, 'rating'] = 10


test_neg = read_file('./neg_test_reviews.csv')
test_neg['sent'] = 0

test = test_pos.append(test_neg)

In [9]:
train_data = list(train.review)
test_data = list(test.review)

In [13]:
tfidf = TfidfVectorizer(stop_words='english',
                       strip_accents='unicode',
                        # ngram_range=(1,2),
                       max_df=0.8
                       )
cv = CountVectorizer(stop_words=stopwords,
                       strip_accents='unicode',
                        # ngram_range=(1,2),
                       # max_df=0.7
                       )

In [16]:
X = cv.fit_transform(train_data)
y = train.sent

In [17]:
X_test = cv.transform(test_data)
y_test = test.sent

In [ ]:
X

In [18]:
nb = MultinomialNB(alpha=0.3)

In [19]:
nb.fit(X, y)

/usr/local/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)


MultinomialNB(alpha=0.3, class_prior=None, fit_prior=True)

In [20]:
print(classification_report(nb.predict(X_test), y_test))

             precision    recall  f1-score   support

          0       0.88      0.78      0.83     13981
          1       0.76      0.86      0.81     11019

avg / total       0.82      0.82      0.82     25000



In [ ]:
from sklearn.svm import SVC


In [ ]:
svc = SVC()

In [460]:
import itertools as it
import operator as op
from sklearn.feature_extraction import DictVectorizer

In [202]:

def get_word_set(filename):
    """
    Gets the dictionary from our preprocessed dataset
    """
    with open (filename, 'r') as f:
        tmp = f.readlines()
        tmp = [line.rstrip() for line in tmp]   
    word_split = lambda x: x.split()[2].split(':')[0]
    return set(map(word_split, t))


def split_list(x):
    t = x.split('\t')
    i = eval(t[0])[0]
    rating = eval(t[0])[1]
    word, count = t[1].split(':')
    return i, rating, word, int(count)
    


In [764]:
def read_file(filename):
 
    reviews = []

    with open(filename, 'r') as f:
        t = f.readlines()
    for line in t:
        ids, rating, review = line.split(',', 2)
        reviews.append([ids, rating, review.strip().lower()])
    df = pd.DataFrame(reviews, columns=['id', 'rating', 'review'])
    df['rating'] = df.rating.astype("int")
    return df
    

def data_maker(_data):
    """
    hacky method to pivot the data from 'long' to 'wide' for each review.
    Args:
        _data: unzipped group-by'd list of stuff
    Returns:
        list of dictionaries with k: v s.t. k = word, v = count(word). 
    """
    res = []
    for row in _data:
        z = {}
        for line in row:
            k, v = line[1].split(':')
            z[k] = int(v)
        res.append(z)
    return res


def read_file(filename):
    """
    preprocesses some of the weirdness out of the format we saved to
    Returns:
        list of all lines in the files according to: [(id, rating), 'word:<count>']
    """
    with open (filename, 'r') as f:
        tmp = f.readlines()
        tmp = [line.rstrip() for line in tmp]
        split = lambda x: x.split('\t')
        tmp = list(map(split, tmp))
        tup, val = list(zip(*tmp))
        # this bit changes all the string tuples to python tuples. 
        tmp = list(zip(map(eval, tup), val))
    return tmp


def make_train_data(data):
    """ 
    Uses Sklearn's helper method to vectorize a list of dictionary features into a scipy 
    sparse matrix for training
    Returns:
        tuple of data, sparse_matrix
    """
    dv = DictVectorizer()
    _data = dict(zip(ids, res))
    return _data, dv.fit_transform(data)
    

get_neg_ids = lambda x: x[0][1] < 5
get_pos_ids = lambda x: x[0][1] > 5

# gets a list of sentiment scores
get_sent = lambda x: 1 if x[1] > 5 else 0

In [779]:
# raw_train = read_file('./train_mr_output.csv')
grouped = it.groupby(raw_train, op.itemgetter(0))

train_g = [(k, list(g)) for k, g in grouped]
ids, _tmp = zip(*train_g)

train = data_maker(_tmp)
train, X = make_train_data(train)
y = list(map(get_sent, ids))



In [780]:
# raw_test = read_file('./test_mr_output.csv')
test_grouped = it.groupby(raw_test, op.itemgetter(0))

test_g = [(k, list(g)) for k, g in test_grouped]
test_ids, test_tmp = zip(*test_g)

test = data_maker(test_tmp)
test, X_test = make_train_data(test)
y_test = list(map(get_sent, test_ids))



In [796]:
X[:,2].data


array([ 5.,  1.,  1.,  2.,  1.,  1.,  2.,  1.,  1.,  1.,  1.,  2.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  2.,  1.,  2.,  1.,  2.,  1.,  1.,  5.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  2.,  1.,  1.,  2.,  1.,  1.,  1.,  1.,  1.,  2.,  1.,  1.,
        1.,  1.,  1.,  2.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  2.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  4.,  1.,  1.,  1.,  2.,  1.,  1.,
        1.,  2.,  1.,  1.,  1.,  1.,  1.,  1.,  4.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  2.,  2.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1

In [782]:
X_test

<25000x74830 sparse matrix of type '<class 'numpy.float64'>'
	with 2464517 stored elements in Compressed Sparse Row format>

In [723]:
grouped = it.groupby(raw_train, op.itemgetter(0))

# g = [(k, list(g)) for k, g in grouped]
ids, _tmp = zip(*g)

train = data_maker(_tmp)
train, X = make_train_data(train)
y = list(map(get_sent, ids))


In [736]:
len(ids)

25000

In [731]:
nb = MultinomialNB(alpha=0.3)

In [739]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)

In [743]:
nb.fit(X, y)

/usr/local/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)


MultinomialNB(alpha=0.3, class_prior=None, fit_prior=True)

In [741]:
sgd.fit(X, y)

/usr/local/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)


SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=42, shuffle=True, verbose=0,
       warm_start=False)

In [772]:
print(classification_report(nb.predict(X_test), y_test))
print(classification_report(sgd.predict(X_test), y_test))

ValueError: dimension mismatch

In [761]:
from sknn.mlp import Classifier, Layer

nn = Classifier(
    layers=[
        Layer("Rectifier", units=10, pieces=2),
        Layer("Softmax")],
    learning_rate=0.001,
    n_iter=25)
